# import and load data set

In [25]:
import geopandas as gpd
import pandas as pd
import sklearn.model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
import numpy as np


from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score

change_type_map = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5}

## Read csvs

train_df = gpd.read_file('train.geojson', index_col=0)
test_df = gpd.read_file('test.geojson', index_col=0)

C:\Users\Ely Cheikh Abass\Documents\Cours\Kaggle_ML\area_classification\.venv\Lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: driver GeoJSON does not support open option INDEX_COL
  return ogr_read(
C:\Users\Ely Cheikh Abass\Documents\Cours\Kaggle_ML\area_classification\.venv\Lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: driver GeoJSON does not support open option INDEX_COL
  return ogr_read(


In [27]:
## Filtering column "mail_type"
## Adding total construction time
import pandas as pd

PRE_CONSTRUCTION_STATUSES = {
    "Greenland",
    "Land Cleared",
    "Prior Construction",
    "Excavation",
    "Materials Dumped",
    "Materials Introduced",
    "Construction Started",
}

POST_CONSTRUCTION_STATUSES = {
    "Construction Midway",
    "Construction Done",
    "Operational",
}


def add_geometry_features(gdf, metric_epsg=6933):
    """
    gdf: GeoDataFrame en EPSG:4326
    Ajoute:
      - polygon_area_m2
      - polygon_perimeter_m
      - compactness (sans unité)
    """
    # 1) reprojection en CRS métrique (mètres)
    print(gdf.geometry.area.isna().sum())
    # print(gdf.geometry.length.isna().sum())
    # print(gdf.geometry.area.isna().sum())
    gdf_m = gdf.to_crs(epsg=metric_epsg, inplace=False)
    print(gdf.geometry.convex_hull.isna().sum())
    # print(gdf.geometry.area[gdf_m.geometry.area.isna()])
    # 2) calculs géométriques en mètres
    gdf = gdf_m.copy()
    gdf["polygon_area_m2"] = gdf.geometry.area
    gdf["polygon_perimeter_m"] = gdf.geometry.length

    # 3) compacité = 4πA / P²
    gdf["compactness"] = (
        4 * np.pi * gdf["polygon_area_m2"] / (gdf["polygon_perimeter_m"] ** 2)
    )
    # gdf.loc[gdf["polygon_perimeter_m"] == 0, "compactness"] = np.nan

    return gdf

def add_max_gap_between_sets(
    df,
    date_cols,     # ex: ["date0", ..., "date4"]
    status_cols    # ex: ["change_status_date0", ..., "change_status_date4"]
):
    out = df.copy()

    dates = out[date_cols].apply(pd.to_datetime, errors="coerce", utc=True)
    statuses = out[status_cols]

    def row_max_gap(row_dates, row_statuses):
        set1_dates = []
        set2_dates = []

        for d, s in zip(row_dates, row_statuses):
            if pd.isna(d) or pd.isna(s):
                continue
            if s in PRE_CONSTRUCTION_STATUSES:
                set1_dates.append(d)
            elif s in POST_CONSTRUCTION_STATUSES:
                set2_dates.append(d)

        if not set1_dates or not set2_dates:
            return 0.0

        # max |t2 - t1|
        max_gap = max(
            abs((t2 - t1).total_seconds())
            for t1 in set1_dates
            for t2 in set2_dates
        )

        return max_gap / 86400.0  # en jours

    out["pre_post_construction_time_gap_days"] = [
        row_max_gap(d, s)
        for d, s in zip(dates.to_numpy(), statuses.to_numpy())
    ]

    return out

date_cols = [f"date{i}" for i in range(5)]
status_cols = [f"change_status_date{i}" for i in range(5)]

train_df = add_geometry_features(train_df)
test_df = add_geometry_features(test_df)


train_df = add_max_gap_between_sets(train_df, date_cols, status_cols)
test_df  = add_max_gap_between_sets(test_df, date_cols, status_cols)
print(','.join(str(train_df[e].isna().sum()) for e in ['polygon_area_m2', 'polygon_perimeter_m', 'compactness']))
print(train_df.crs)

#
# date_cols = [f'date{i}' for i in range(5)]
# status_cols = [f'change_status_date{i}' for i in range(5)]
#
# def get_const_time(df):
#     dt_cols = []
#     for i, c in enumerate(date_cols):
#         dtc = f"dt_{i}"
#         df[dtc] = pd.to_datetime(df[c], errors="coerce", utc=True)  # utc pour être stable
#         dt_cols.append(dtc)
#
#     for i, c in enumerate(status_cols):
#         df[f"change_status_date{i}"].apply(lambda x: status_cols[x])

C:\Users\Ely Cheikh Abass\Documents\Cours\Kaggle_ML\area_classification\.venv\Lib\site-packages\shapely\measurement.py:50: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)


48


GEOSException: IllegalArgumentException: CGAlgorithmsDD::orientationIndex encountered NaN/Inf numbers

In [3]:

# train_x_d = np.asarray(train_df['date0'])
feature_cols = [
    "pre_post_construction_time_gap_days",
    "polygon_area_m2",
    "polygon_perimeter_m",
    "compactness",
]

# X (numpy arrays)
train_x = train_df[feature_cols].to_numpy(dtype=float)
test_x  = test_df[feature_cols].to_numpy(dtype=float)

# y (labels encodés)
train_y = train_df["change_type"].map(change_type_map).to_numpy()

print(train_x.shape, train_y.shape, test_x.shape)

(296146, 4) (296146,) (120526, 4)


In [4]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold

# =========================
# PARAMÈTRES
# =========================
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# pipeline FIXE : Imputer -> Scaler -> SVC
svc_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),  # gère NaN
    ("scaler", StandardScaler()),                   # SVC a besoin de scaling
    ("model", SVC(kernel="rbf", C=1))
])

# =========================
# FEATURES AVANT (baseline)
# =========================
baseline_features = [
    "polygon_area_m2"   # <-- mets EXACTEMENT ce que tu utilisais avant
]

X_old = train_df[baseline_features].to_numpy(dtype=float)
y = train_df["change_type"].map(change_type_map).to_numpy()

scores_old = cross_val_score(
    svc_pipe, X_old, y,
    cv=cv, scoring="f1_macro", n_jobs=-1
)

# =========================
# FEATURES APRÈS (nouvelles)
# =========================
new_features = [
    "pre_post_construction_time_gap_days",
    "polygon_area_m2",
    "polygon_perimeter_m",
    "compactness",
]

X_new = train_df[new_features].to_numpy(dtype=float)

scores_new = cross_val_score(
    svc_pipe, X_new, y,
    cv=cv, scoring="f1_macro", n_jobs=-1
)

# =========================
# RÉSULTATS
# =========================
print("===== AVANT =====")
print("F1 macro mean :", scores_old.mean())
print("F1 macro std  :", scores_old.std())

print("\n===== APRÈS =====")
print("F1 macro mean :", scores_new.mean())
print("F1 macro std  :", scores_new.std())

print("\n===== GAIN =====")
print("Gain absolu :", scores_new.mean() - scores_old.mean())
print("Gain relatif (%) :", (scores_new.mean() / scores_old.mean() - 1) * 100)


===== AVANT =====
F1 macro mean : 0.12368922830162718
F1 macro std  : 0.00516401722936706

===== APRÈS =====
F1 macro mean : nan
F1 macro std  : nan

===== GAIN =====
Gain absolu : nan
Gain relatif (%) : nan


C:\Users\Ely Cheikh Abass\Documents\Cours\Kaggle_ML\area_classification\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:490: FitFailedWarning: 
4 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Ely Cheikh Abass\Documents\Cours\Kaggle_ML\area_classification\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 833, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Ely Cheikh Abass\Documents\Cours\Kaggle_ML\area_classification\.venv\Lib\site-packages\sklearn\base.py", line 1336, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^

In [5]:
# =========================
# GRIDSEARCH "PROPRE & FORT"
# - gère les NaN (imputer)
# - compare plusieurs algos
# - optimise F1 macro (Kaggle metric)
# =========================

import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# ---------- 1) CHOISIS TES FEATURES ----------
feature_cols = [
    "pre_post_construction_time_gap_days",
    "polygon_area_m2",
    "polygon_perimeter_m",
    "compactness",
]

X = train_df[feature_cols].to_numpy(dtype=float)
y = train_df["change_type"].map(change_type_map).to_numpy()

# ---------- 2) CV + PIPELINE BASE ----------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),  # gère NaN pour TOUS les modèles
    ("scaler", StandardScaler()),                   # utile pour SVC/LogReg, neutre si passthrough
    ("model", LogisticRegression())                 # placeholder, remplacé par GridSearch
])

# ---------- 3) GRILLES (une par modèle) ----------
param_grid = [
    # ---- SVC RBF (souvent très bon si features OK) ----
    {
        "scaler": [StandardScaler()],
        "model": [SVC(kernel="rbf")],
        "model__C": [0.3, 1, 3, 10],
        "model__gamma": ["scale", "auto"],
        "model__class_weight": [None, "balanced"],
    },

    # ---- Logistic Regression (baseline solide) ----
    {
        "scaler": [StandardScaler()],
        "model": [LogisticRegression(max_iter=5000, n_jobs=None)],
        "model__C": [0.1, 1, 10],
        "model__class_weight": [None, "balanced"],
        "model__solver": ["lbfgs"],
        "model__multi_class": ["auto"],
    },

    # ---- RandomForest (robuste, pas sensible au scaling) ----
    {
        "scaler": ["passthrough"],
        "model": [RandomForestClassifier(random_state=42, n_jobs=1)],
        "model__n_estimators": [400, 800],
        "model__max_depth": [None, 12, 24],
        "model__min_samples_split": [2, 8],
        "model__min_samples_leaf": [1, 3],
        "model__class_weight": [None, "balanced_subsample"],
    },

    # ---- ExtraTrees (souvent encore meilleur que RF) ----
    {
        "scaler": ["passthrough"],
        "model": [ExtraTreesClassifier(random_state=42, n_jobs=1)],
        "model__n_estimators": [600, 1200],
        "model__max_depth": [None, 18, 30],
        "model__min_samples_split": [2, 8],
        "model__min_samples_leaf": [1, 3],
        "model__class_weight": [None, "balanced"],
    },

    # ---- AdaBoost (à tester, parfois très bon, parfois sensible au bruit) ----
    {
        "scaler": ["passthrough"],
        "model": [AdaBoostClassifier(
            estimator=DecisionTreeClassifier(max_depth=2, random_state=42),
            random_state=42
        )],
        "model__n_estimators": [200, 400],
        "model__learning_rate": [0.05, 0.2, 0.6],
    },

    # ---- HistGradientBoosting (très fort en tabulaire; gère bien) ----
    {
        "scaler": ["passthrough"],
        "model": [HistGradientBoostingClassifier(random_state=42)],
        "model__learning_rate": [0.03, 0.1],
        "model__max_depth": [None, 6, 12],
        "model__max_leaf_nodes": [31, 63, 127],
        "model__min_samples_leaf": [10, 30],
    },
]

# ---------- 4) GRIDSEARCH ----------
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=2,
    refit=True,
    return_train_score=False
)

grid.fit(X, y)

# ---------- 5) RÉSULTATS ----------
print("\n===== BEST =====")
print("best f1_macro:", grid.best_score_)
print("best params:", grid.best_params_)
print("best model:", grid.best_estimator_)

# ---------- 6) BEST MODEL prêt pour train/test ----------
best_model = grid.best_estimator_

# (option) fit sur tout le train (déjà fait par refit=True)
# best_model.fit(X, y)

# (option) prédire test (assure-toi que test_df a les mêmes features + mêmes noms)
# X_test = test_df[feature_cols].to_numpy(dtype=float)
# y_test_pred = best_model.predict(X_test)


Fitting 5 folds for each of 160 candidates, totalling 800 fits


ValueError: Invalid parameter 'multi_class' for estimator LogisticRegression(C=0.1, max_iter=5000). Valid parameters are: ['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'].

# Predict and create csv prediction

In [ ]:
pred_y = grid.predict(test_x)
print(f"{accuracy_score(train_y, pred_y)}")

In [ ]:

## Save results to submission file
pred_df = pd.DataFrame(pred_y, columns=['change_type'])
pred_df.to_csv("knn_sample_submission.csv", index=True, index_label='Id')